In [9]:
import math
import numpy as np
import cv2

In [16]:
def custom_top_hat_filter(image, kernel_size):
    if len(image.shape) == 3:
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray_image = image

    kernel = np.ones(kernel_size, np.uint8)
    eroded = cv2.erode(gray_image, kernel, iterations=1)
    top_hat = gray_image - eroded

    return top_hat

In [10]:
def calculate_center(rect):
    x, y, w, h = rect
    cx = (x + w) // 2
    cy = (y + h) // 2
    return cx, cy


In [11]:
def update(objects_rect, center_points, id_count):
    objects_bbs_ids = []
    new_center_points = {}

    for rect in objects_rect:
        cx, cy = calculate_center(rect)

        same_object_detected = False

        for id, pt in center_points.items():
            dist = math.hypot(cx - pt[0], cy - pt[1])

            if dist < 25:
                center_points[id] = (cx, cy)
                objects_bbs_ids.append([rect[0], rect[1], rect[2], rect[3], id])
                same_object_detected = True
                break

        if not same_object_detected:
            center_points[id_count] = (cx, cy)
            objects_bbs_ids.append([rect[0], rect[1], rect[2], rect[3], id_count])
            id_count += 1

    for obj_bb_id in objects_bbs_ids:
        _, _, _, _, object_id = obj_bb_id
        center = center_points[object_id]
        new_center_points[object_id] = center

    center_points = new_center_points.copy()

    return objects_bbs_ids, center_points, id_count


In [19]:
import cv2

# Mở video từ tệp "highway.mp4"
cap = cv2.VideoCapture("highway.mp4")

# Khởi tạo bộ phát hiện đối tượng từ camera ổn định
object_detector = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=40)

# Khởi tạo danh sách để lưu trữ tâm của các đối tượng đã được theo dõi và biến đếm ID
center_points = {}
id_count = 0

# Bắt đầu vòng lặp chạy qua video
while True:
    # Đọc frame từ video
    ret, frame = cap.read()
    if not ret:
        break  # Thoát khỏi vòng lặp nếu hết video

    # Lấy kích thước của frame
    height, width, _ = frame.shape

    # Cắt ra vùng quan tâm (Region of Interest - ROI)
    roi = frame[340: 720, 500: 800]

    # Bước 1: Phát hiện đối tượng
    mask = object_detector.apply(roi)
    _, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    detections = []
    for cnt in contours:
        # Tính diện tích và loại bỏ các thành phần nhỏ
        area = cv2.contourArea(cnt)
        if area > 100:
            x, y, w, h = cv2.boundingRect(cnt)
            detections.append([x, y, w, h])

    # Bước 2: Theo dõi đối tượng
    boxes_ids, center_points, id_count = update(detections, center_points, id_count)

    for box_id in boxes_ids:
        x, y, w, h, id = box_id
        cv2.putText(roi, str(id), (x, y - 15), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 2)
        cv2.rectangle(roi, (x, y), (x + w, y + h), (0, 255, 0), 3)

    # Hiển thị vùng quan tâm (ROI), khung hình (frame) và mặt nạ (mask)
    cv2.imshow("roi", roi)
    cv2.imshow("Frame", frame)
    cv2.imshow("Mask", mask)

    key = cv2.waitKey(30)
    if key == 27:
        break

# Giải phóng tài nguyên và đóng cửa sổ hiển thị
cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

In [1]:
import cv2

# Tạo ba bộ phân đoạn nền sử dụng ba thuật toán khác nhau
fgbg2 = cv2.createBackgroundSubtractorMOG2()

# Mở video từ tệp tin (thay 'ten_file_video.mp4' bằng đường dẫn đến tệp video của bạn)
cap = cv2.VideoCapture('../../object_tracking/highway.mp4')
out_path = "./images"
frame_number = 0  # Biến đếm số frame

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Áp dụng mask cho phân đoạn nền
    fgmask2 = fgbg2.apply(frame)

    # Lưu frame thành tệp ảnh
    cv2.imwrite(f'{out_path}/frame_{frame_number:04d}.png', frame)
    frame_number += 1

    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

cap.release()

cv2.destroyAllWindows()

In [ ]:
import os

folder_path = "./images"  # Thay đổi đường dẫn đến thư mục của bạn

for filename in os.listdir(folder_path):
    if filename.endswith(".jpg"):
        file_path = os.path.join(folder_path, filename)
        os.remove(file_path)